## Reading the data from LocalFile

Alternatively to reading the data from the database you can read it form the CSV file at this [path](../data/2024-11-06-news.csv) and load it in a dataframe named `news_df`.


In [ ]:
import pandas as pd

In [ ]:
from pathlib import Path
current_directory = Path.cwd()

In [ ]:
data_path = current_directory.joinpath("data", "2025-10-9-news.csv")

In [ ]:
news_df = pd.read_csv(data_path, index_col=0, sep='|')

In [ ]:
news_df.shape

In [ ]:
print(len(news_df.sample(1).content.values[0].split("\nn")))

In [ ]:
news_df.head()

Things I need to do:

- Perform the cluster on tile and and paragraph(chunks)

- For each cluster generates questions and answers that can be answers by it.
- Improve summarization and then do evaluation of the summaries using summarization evaluation metrics.

In [ ]:
content_paragraphs = news_df.content.str.strip().str.split("\nn").explode().to_frame()

In [ ]:
content_length = content_paragraphs.content.str.strip().str.len()

Get content of more than 50 characters.

In [ ]:
content_paragraphs = content_paragraphs.loc[content_length > 50]

In [ ]:
content_paragraphs

In [ ]:
content_paragraphs = content_paragraphs.merge(news_df[['title']], how='inner', left_index=True, right_index=True)

In [ ]:
chunks_df= content_paragraphs.content.str.cat(content_paragraphs.title, sep="\n\n").to_frame()

In [ ]:
chunks_df["title"] = content_paragraphs.title

In [ ]:
chunks_df = chunks_df.reset_index(drop=True)
chunks_df.head()

The bellow phase I have split each text in different paragraphs, and then I considered each paragraph as a chunk.

On top of each paragraph I added the title of the post where it was found.

## 2. Embedding phase.




In [ ]:
embedding_model_id = "dunzhang/stella_en_400M_v5"

In [ ]:
current_directory

In [ ]:
model_path = current_directory.joinpath(embedding_model_id)

In [ ]:
embedding_model_path = current_directory.joinpath("models", embedding_model_id)

In [ ]:
from src.summarizer.cluster_modeler import HierarchicalClusterModeler
from src.summarizer.data_puller import DataPuller
from src.summarizer.embeddings_computer import EmbeddingsComputer

In [ ]:
embedding_modeller = EmbeddingsComputer(embedding_model_id=embedding_model_id)

In [ ]:
chunks_df.shape

In [ ]:
embedding_documents = embedding_modeller.run(
    documents=chunks_df["content"])

In [ ]:
cluster_modeler = HierarchicalClusterModeler()

In [ ]:
important_news_df = cluster_modeler.run(
    today_news_embeddings=embedding_documents, documents=chunks_df
)

In [ ]:
important_news_df.query("labels == 28").title.unique()

i

In [ ]:
important_news_df.labels.unique()

Ohh let do GenAI,

Let stop investing in Machine Learning and Invest in GenAI

1 year later: 

https://www.theregister.com/2025/08/18/generative_ai_zero_return_95_percent/